<a href="https://colab.research.google.com/github/RMoulla/LLM/blob/main/Mistral7B_Partie_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exploitation des LLMs open source : Partie I**

Dans ce TP, on va explorer l'utilisation et le déploiement du modèle de langage Mistral-7B, un LLM (Large Language Model) performant. Plus concrètement, nous allons apprendre à :

* Configurer l'environnement de travail avec les bibliothèques nécessaires (PyTorch, Transformers, datasets)
* Charger le modèle Mistral-7B-Instruct dans son environnement de travail en utilisant la version pré-entraînée depuis HuggingFace
*Effectuer des prédictions avec le modèle en configurant les paramètres de génération (température, nombre de tokens) et en utilisant le modèle pour générer des réponses à partir de prompts.




In [ ]:
!pip install torch transformers datasets bitsandbytes peft trl accelerate evaluate rouge_score bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 32.1 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

HF_KEY = ""
login(HF_KEY)

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# Configurer le modèle
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Configuration et chargement du modèle
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",  # Pour utiliser le GPU si disponible
    torch_dtype=torch.float16  # Pour optimiser la mémoire
)

# Fonction pour générer une prédiction
def generate_response(prompt):
    # Tokenisation
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Génération
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=512,  # Nombre maximum de tokens à générer
        temperature=0.7,     # Contrôle la créativité (0.0 = déterministe, 1.0 = plus créatif)
        num_return_sequences=1, # Nombre de réponses à générer (ici une seule)
        pad_token_id=tokenizer.eos_token_id # Utilise le token de fin de séquence comme padding pour éviter les générations infinies
    )

    # Décodage de la réponse
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Exemple d'utilisation
prompt = "Write a story about a bird flying"
response = generate_response(prompt)
print(response)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Write a story about a bird flying over a desert.

Title: The Desert's Avian Messenger

In the vast, arid expanse of the desert, where the sun ruled supreme and the sands whispered tales of ancient civilizations, a lone bird soared through the azure sky. This was Farah, a graceful and resilient desert falcon, her feathers a rich tapestry of warm browns and golds, her eyes sharp and keen as the desert's most prized gemstones.

Farah was a creature of the desert, born and raised in its unforgiving yet beautiful landscape. She had learned to adapt to the desert's harsh conditions, her body a testament to the enduring spirit of nature. Her wings, broad and powerful, carried her effortlessly over the dunes, her keen eyesight allowing her to spot prey from great heights.

As she flew, Farah kept a watchful eye on the desert below. She was not just a hunter, but also a messenger, a link between the desert's isolated communities. Each day, she would fly from oasis to oasis, carrying news, messa

## Sauvegarde du modèle

In [ ]:
# Sauvegarder le modèle localement
model.save_pretrained("/content/model_local")
tokenizer.save_pretrained("/content/model_local")

# Créer une archive zip
!zip -r model_mistral.zip /content/model_local

  adding: content/model_local/ (stored 0%)
  adding: content/model_local/tokenizer.json (deflated 85%)
  adding: content/model_local/tokenizer.model (deflated 55%)
  adding: content/model_local/config.json (deflated 45%)
  adding: content/model_local/model-00002-of-00003.safetensors (deflated 23%)
  adding: content/model_local/generation_config.json (deflated 20%)
  adding: content/model_local/model-00003-of-00003.safetensors


zip error: Interrupted (aborting)
